In [4]:
# API - Método Wanderley de Investimentos (MWI)

"""
    Esta é uma aplicação que busca automatizar a  análise de investimentos de ações de empresas na 
    bolsa de valores brasileira, utilizando a base de dados do site Status Invest e referenciando
    os critérios pessoais do usuário, a saber:
    - DY > 6%
    - Margens > 10%
    - Rentabilidade > 10%
    - Dívida/EBITDA < 2
    - Liquidez > 1M
    - Preço < Graham
    - Graham/Preço > 10%
"""

# EXTRAÇÃO DOS DADOS

from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep

url = "https://statusinvest.com.br/acoes/busca-avancada"

navegador = webdriver.Edge()
navegador.get(url)

sleep(2)

botão_buscar = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[3]/div/div/div/button[2]')
botão_buscar.click()

sleep(2)

botão_download = navegador.find_element(By.XPATH, '/html/body/main/div[4]/div/div[1]/div[2]/a')
botão_download.click()

sleep(2)

navegador.quit()

ModuleNotFoundError: No module named 'selenium'

In [6]:
# API - Método Wanderley de Investimentos (MWI)

"""
    Esta é uma aplicação que busca automatizar a  análise de investimentos de ações de empresas na 
    bolsa de valores brasileira, utilizando a base de dados do site Status Invest e referenciando
    os critérios pessoais do usuário, a saber:
    - DY > 6%
    - Margens > 10%
    - Rentabilidade > 10%
    - Dívida/EBITDA < 2
    - Liquidez > 1M
    - Preço < Graham
    - Graham/Preço > 10%
"""

# TRATAMENTO DOS DADOS

from csv import writer
from csv import DictWriter
from numpy import sqrt

arquivo = "statusinvest-busca-avancada.csv"

dados = []

with open(arquivo) as arquivo:
    # REALIZANDO A SEPARAÇÃO DOS DADOS
    dados = arquivo.read().strip().split('\n')
    cabeçalho = dados[0].strip().split(';')
    status_invest = []
    for dado in range(1, len(dados)):
        indicadores = dados[dado].strip().replace('.','').replace(',', '.').split(';')
        empresa = {}
        for indicador in range(len(indicadores)):
            if indicador == 0:
                empresa[cabeçalho[indicador]] = indicadores[indicador]
            else:
                try:
                    empresa[cabeçalho[indicador]] = float(indicadores[indicador])
                except ValueError:
                    empresa[cabeçalho[indicador]] = 0.0
        status_invest.append(empresa)

print(f"Número total de empresas analisadas: {len(status_invest)}\n")

qtdd_empresas = len(status_invest)

# ANÁLISE DOS DADOS - DY > 6% (Método de Bazin)

def dy(status_invest):
    for n in range(len(status_invest)):
        try:
            if status_invest[n]["DY"] < 6:
                del status_invest[n]
                dy(status_invest)
        except IndexError:
            pass

dy(status_invest)

print(f"Método de Bazin \nNúmero de empresas eliminadas: {qtdd_empresas - len(status_invest)}")

qtdd_empresas = len(status_invest)

# ANÁLISE DOS DADOS - MARGENS > 10%

def margens(status_invest):
    for n in range(len(status_invest)):
        try: 
            if status_invest[n]["MARGEM BRUTA"] < 10:
                del status_invest[n]
                margens(status_invest)
            if status_invest[n]["MARGEM EBIT"] < 10:
                del status_invest[n]
                margens(status_invest)
            if status_invest[n]["MARG. LIQUIDA"] < 10:
                del status_invest[n]
                margens(status_invest)
        except IndexError:
            pass

margens(status_invest)

print(f"Margens > 10% \nNúmero de empresas eliminadas: {qtdd_empresas - len(status_invest)}")

qtdd_empresas = len(status_invest)

# ANÁLISE DOS DADOS - RENTABILIDADES > 10%

def rentabilidade(status_invest):
    for n in range(len(status_invest)):
        try: 
            if status_invest[n]["ROE"] < 10:
                del status_invest[n]
                rentabilidade(status_invest)
            if status_invest[n]["ROA"] < 10:
                del status_invest[n]
                rentabilidade(status_invest)
            if status_invest[n]["ROIC"] < 10:
                del status_invest[n]
                rentabilidade(status_invest)
        except IndexError:
            pass

rentabilidade(status_invest)

print(f"Rentabilidades > 10% \nNúmero de empresas eliminadas: {qtdd_empresas - len(status_invest)}")

qtdd_empresas = len(status_invest)

# ANÁLISE DOS DADOS - DÍVIDAS/EBIT < 2

def divida(status_invest):
    for n in range(len(status_invest)):
        try:
            if status_invest[n]["DIVIDA LIQUIDA / EBIT"] > 2:
                del status_invest[n]
                divida(status_invest)
        except IndexError:
            pass

divida(status_invest)

print(f"Dívida/EBIT < 2 \nNúmero de empresas eliminadas: {qtdd_empresas - len(status_invest)}")

qtdd_empresas = len(status_invest)

# ANÁLISE DE DADOS - LIQUIDEZ > 1M

def liquidez(status_invest):
    for n in range(len(status_invest)):
        try:
            if status_invest[n][" LIQUIDEZ MEDIA DIARIA"] < 1_000_000:
                del status_invest[n]
                liquidez(status_invest)
        except IndexError:
            pass

liquidez(status_invest)

print(f"Liquidez > 1M \nNúmero de empresas eliminadas: {qtdd_empresas - len(status_invest)}")

qtdd_empresas = len(status_invest)

# ANÁLISE DE DADOS - GRAHAM > PREÇO (Método de Graham)

def indice_graham(status_invest):
    for n in range(len(status_invest)):
        try:
            graham = sqrt([22.5 * status_invest[n][" VPA"] * status_invest[n][" LPA"]])
            if graham < 1.1 * status_invest[n]["PRECO"]:
                del status_invest[n]
                indice_graham(status_invest)
        except IndexError:
            pass

indice_graham(status_invest)

print(f"Método de Graham \nNúmero de empresas eliminadas: {qtdd_empresas - len(status_invest)}\n")

# ANÁLISE DE DADOS - Definindo proporção da carteira
graham_max = 0
for z in range(len(status_invest)):
    graham = sqrt([22.5 * status_invest[z][" VPA"] * status_invest[z][" LPA"]])
    if graham > graham_max:
        graham_max = graham

empresas_selecionadas = []

for i in range(len(status_invest)):
    qtdd = int(graham_max // status_invest[i]["PRECO"])
    empresas_selecionadas.append(str(qtdd) + status_invest[i]["TICKER"])

print(f"\n{empresas_selecionadas}\n")

Número total de empresas analisadas: 604

Método de Bazin 
Número de empresas eliminadas: 494
Margens > 10% 
Número de empresas eliminadas: 50
Rentabilidades > 10% 
Número de empresas eliminadas: 34
Dívida/EBIT < 2 
Número de empresas eliminadas: 3
Liquidez > 1M 
Número de empresas eliminadas: 7
Método de Graham 
Número de empresas eliminadas: 6


['27FIQE3', '9ITSA3', '9ITSA4', '10KEPL3', '12LAVV3', '17POMO3', '13SYNE3', '4TGMA3', '1VALE3', '4VLID3']



C:\Users\Daniel\AppData\Local\Temp\ipykernel_13512\2280521695.py:171: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  qtdd = int(graham_max // status_invest[i]["PRECO"])
